# Intro to the linear algebra module
Most of the linear algebra module are wrappers with very few lines and an API nearly equal to their numpy counterpart. The only thing you need to do is pass the input DataArray and indicate which dimensions
correspond to the matrices.

In [1]:
from xarray_einstats import linalg, tutorial

We start by generating syntetic data to work with:

In [2]:
da = tutorial.generate_matrices_dataarray(7)
da

<xarray.DataArray (batch: 10, experiment: 3, dim: 4, dim2: 4)>
0.7075 1.025 0.5685 0.8951 0.2065 3.384 ... 1.239 0.4527 0.5749 0.4766 0.859
Dimensions without coordinates: batch, experiment, dim, dim2

In this case, the data represents a collection of matrices. `dim` and `dim2` indicate the matrix dimensions, the whole array is 4d, with 30 matrices in total from 10 batches and 3 experiments. You can get the trace of all 30 matrices in a single line.

In [3]:
linalg.trace(da, dims=["dim", "dim2"])

<xarray.DataArray (batch: 10, experiment: 3)>
4.854 4.74 4.457 2.637 2.79 3.163 1.998 ... 2.804 4.58 2.888 4.936 5.983 4.07
Dimensions without coordinates: batch, experiment

The main feature of the wrappers is that they know what is the expected shape of the output, you don't need to take care of it. See how the inverse which doesn't reduce the matrix dimension can be called with the exact same arguments.

In [4]:
linalg.inv(da, dims=["dim", "dim2"])

<xarray.DataArray (batch: 10, experiment: 3, dim: 4, dim2: 4)>
11.26 -2.363 -10.84 -0.2744 10.99 -2.017 ... -3.444 0.7703 0.316 0.01949 -1.162
Dimensions without coordinates: batch, experiment, dim, dim2

Even a qr decomposition which returns multiple matrices (which could even have different shapes) needs only these two arguments to work

In [5]:
q, r = linalg.qr(da, dims=["dim", "dim2"])

In [6]:
q

<xarray.DataArray (batch: 10, experiment: 3, dim: 4, dim2: 4)>
-0.5452 0.01652 -0.5624 -0.6214 -0.1592 ... -0.3322 -0.4013 0.2607 0.8128
Dimensions without coordinates: batch, experiment, dim, dim2

In [7]:
r

<xarray.DataArray (batch: 10, experiment: 3, dim: 4, dim2: 4)>
-1.298 -1.975 -1.858 -1.228 0.0 -3.137 ... -0.4307 1.052 0.0 0.0 0.0 -0.6995
Dimensions without coordinates: batch, experiment, dim, dim2

:::{tip}
Do you always follow the same convention to name your matrix dimensions and feel that even having to repeat that is
unnecessary? Take a look at {func}`xarray.linalg.get_default_dims` to see how to modify the default dims used by the linalg wrappers
:::

## einsum
`einsum` is a such a flexible function that it can even be intimidating. It can cover from `sum` operations, to {func}`xarray.dot` reductions and obviously some operations similar to `einops` which after all is inspired in einsum. 

The goal of this page is not to be an extensive nor in depth guide on einsum but to act as a small ladder from simple operations that you can do without einsum until reaching operations that are only possible with einsum. This will give you a good look into `xarray_einstats` unique version of `einsum` that works with named dimensions, you'll see how most einsum operations translate to our syntax. If you want to master einsum however, we direct you to {func}`numpy.einsum` documentation and the [einops]() package.

In [8]:
from xarray_einstats import raw_einsum, einsum

Start reducing the `experiment` dimension. Any ellipsis, broadcasting and transposition is handled by xarray and xarray-einstats. You only need to care about the dimensions you want to operate on. Use `[]` to indicate you want to reduce the dimension (or `->` in `raw_` syntax):

In [27]:
einsum([["experiment"], []], da)
raw_einsum("experiment->", da)

<xarray.DataArray (batch: 10, dim: 4, dim2: 4)>
4.487 3.158 0.9252 2.683 0.5319 3.799 ... 3.387 1.796 2.601 2.455 1.538 5.402
Dimensions without coordinates: batch, dim, dim2

In [ ]:
einsum([["experiment"], []], da)
raw_einsum("experiment->", da)

In [20]:
da.sum(("batch", "experiment"))

<xarray.DataArray (dim: 4, dim2: 4)>
22.27 32.55 29.06 40.96 23.96 33.48 ... 25.27 29.59 34.97 20.57 34.89 30.26
Dimensions without coordinates: dim, dim2

In [17]:
raw_einsum("draw,draw", ds.a, ds.b, keep_dims={"chain"})

<xarray.DataArray (team: 7, chain: 4, chain2: 4)>
30 30 30 30 30 30 30 30 30 30 30 30 30 ... 30 30 30 30 30 30 30 30 30 30 30 30
Dimensions without coordinates: team, chain, chain2

In [ ]:
np.einsum()